In [1]:
import os
import folium

print(folium.__version__)

0.3.0.dev


In [2]:
m = folium.Map(location=[44, -73], zoom_start=5)

marker_cluster = folium.MarkerCluster().add_to(m)


folium.Marker(
    location=[40.67, -73.94],
    popup='Add popup text here.',
    icon=folium.Icon(color='green', icon='ok-sign'),
).add_to(marker_cluster)

folium.Marker(
    location=[44.67, -73.94],
    popup='Add popup text here.',
    icon=folium.Icon(color='red', icon='remove-sign'),
).add_to(marker_cluster)

folium.Marker(
    location=[44.67, -71.94],
    popup='Add popup text here.',
    icon=None,
).add_to(marker_cluster)

m.save(os.path.join('results', 'MarkerCluster_simple.html'))

m